In [72]:
#todo
#statistics of returns
#regression (?)
#improve charts (titles etc..)
#import financials, possibile in a new "tab"
#import twitter scraping for sentimetn
#additional tab with portfolio (?)

env: ALPHAVANTAGE_API_KEY=NTN3D6P6LIMZ6A2Q#alpha advantage api key
Dash app running on http://127.0.0.1:8050/


In [5]:
#graph imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
#data imports
import pandas as pd
import numpy as np
import os
import pandas_datareader.data as web
from datetime import datetime as dt
from bs4 import BeautifulSoup
import requests
import sys
################################################################################
%env ALPHAVANTAGE_API_KEY = NTN3D6P6LIMZ6A2Q#alpha advantage api key
################################################################################


from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='YOUR_API_KEY',output_format='pandas', indexing_type='date')

################################################################################
#adjust value function
################################################################################
def adjust(date, close, adj_close, in_col, rounding=4):
    '''
    If using forex or Crypto - Change the rounding accordingly!
    '''
    try:
        factor = adj_close / close
        return round(in_col * factor, rounding)
    except ZeroDivisionError:
        print('WARNING: DIRTY DATA >> {} Close: {} | Adj Close {} | in_col: {}'.format(date, close, adj_close, in_col))
        return 0
    
    
def drawText():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H2("Text"),
                ], style={'textAlign': 'center'}) 
            ])
        ),
    ])

app = JupyterDash(external_stylesheets=[dbc.themes.SLATE])

app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    dcc.Input(id = 'stock-ticker', value = 'KO', type = 'text'),
                    html.Button(id = 'submit-ticker', n_clicks = 0, children = 'Submit'),
                    dcc.Store(id='stock-storage')
                ], width=2, align = 'center'),
                dbc.Col([
                    dbc.CardBody([
                        html.Div(id = 'stock-name')    
                    ], style={'textAlign': 'center'}
                    )
                    #html.Div(id='test')
                ], width=2),
                dbc.Col([
                    dbc.CardBody([
                        html.Div(id = 'stock-price')    
                    ], style={'textAlign': 'center'}
                    )
                    #html.Div(id='test')
                ], width=2),
                dbc.Col([
                     html.Br(),
                     html.Div([dcc.DatePickerRange(id="my-date-picker-range",
                                                    min_date_allowed=dt(2000, 1, 1),
                                                    max_date_allowed=dt.today(),
                                                    start_date = dt(2000, 1, 1),
                                                    end_date=dt.today(),
                                                    updatemode='bothdates')
                              ]),
                     html.Br()
                ], width=3)
            ]), 
            #html.Br(),
            dbc.Row([
                dbc.Col([
                    html.Div([
                        dbc.Card(
                            dbc.CardBody([
                               html.Div(id = 'candlestick-graph'),
                            ])
                        ),  
                    ])
                ], width=6),
                dbc.Col([
                    html.Div([
                        dbc.Card(
                            html.Div(id = 'daily-returns-graph'),
                        ),  
                    ])
                ], width=6),
            ], align='center'),
             html.Br(),
            dbc.Row([
                dbc.Col([
                    html.Div([
                        dbc.Card(
                           dbc.CardBody([
                                html.Div(id = 'cum-returns-graph'),
                            ])
                        ),  
                    ])
                ], width=6),
                dbc.Col([
                    html.Div([
                        dbc.Card(
                           dbc.CardBody([
                                html.Div(id = 'daily-returns-time-graph'),
                            ])
                        ),  
                    ])
                ], width=6),
            ], align='center'),
        ]), color = 'dark'
    )
])

@app.callback(Output(component_id='stock-storage', component_property='data'),
              Output(component_id='stock-name', component_property='children'),
              Output(component_id='stock-price', component_property='children'),
              Input('submit-ticker', 'n_clicks'),
              State('stock-ticker', 'value'))
def update_output(n_clicks,stock):
    df, meta_data = ts.get_daily_adjusted(stock, outputsize='full')
    df.sort_index(ascending=True, inplace = True)  
    
    url = "https://finance.yahoo.com/quote/{}".format(stock)

    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')

    price = soup.find('span', class_='Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)')
    name = soup.find('h1', class_ = 'D(ib) Fz(18px)')
        
    return df.to_json(), html.H2(name.text), html.H2("Last price: {}".format(price.text))

@app.callback(Output(component_id='daily-returns-graph' , component_property= 'children'),
              Output(component_id='candlestick-graph', component_property= 'children'),
              Output(component_id='cum-returns-graph' , component_property= 'children'),
              Output(component_id='daily-returns-time-graph',component_property='children'),
              Output(component_id='my-date-picker-range', component_property = 'start_date'),
               Output(component_id='my-date-picker-range', component_property = 'end_date'),
              Input('stock-storage','data') )
def update_chart(stock):
    if stock is not None:
        df = pd.read_json(stock)

        df['adj close'] = df['5. adjusted close']
        df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
        df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
        df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])
    
    
        #calculate daily returns
        df['daily_return'] = df['adj close'].pct_change()
        df['log_returns'] = (np.log(df['adj close'] /    df['adj close'].shift(-1)))
        # calculate cumluative return
        df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000
    
        daily_returns = dcc.Graph(#id='daily-returns-graph',
                                        figure=px.histogram(df, x="daily_return",  title = 'Daily returns distribution'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Returns",
                                            yaxis_title="Frequency",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )   
        candlestick = dcc.Graph(#id='candlestick-graph',
                                        figure=go.Figure(data=[go.Candlestick(x=df.index,
                                                        open=df['adj open'],
                                                        high=df['adj high'],
                                                        low=df['adj low'],
                                                        close=df['adj close'])]
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        cum_returns = dcc.Graph(#id='cum-returns-graph',
                                        figure=px.line(df, x=df.index,y="cumluative_return", title = 'Cumulative returns'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Value of portfolio",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        daily_returns_time = dcc.Graph(#id='daily-returns-time-graph',
                                        figure=px.line(df, x=df.index,y="daily_return", title = 'Daily returns over time'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Returns",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )            
        date_picker = dcc.DatePickerRange(id = 'my-date-picker-range',
                                min_date_allowed=pd.to_datetime(df.index).min(),
                                max_date_allowed=pd.to_datetime(df.index).max(),
                                start_date=pd.to_datetime(df.index).min(),
                                end_date=pd.to_datetime(df.index).max(),
                                updatemode='bothdates')
    
        return daily_returns, candlestick,  cum_returns, daily_returns_time, pd.to_datetime(df.index).min(), pd.to_datetime(df.index).max()
    else:
        return None, None, None, None, None, None


@app.callback(Output(component_id='daily-returns-graph' , component_property= 'children'),
              Output(component_id='candlestick-graph', component_property= 'children'),
              Output(component_id='cum-returns-graph' , component_property= 'children'),
              Output(component_id='daily-returns-time-graph',component_property='children'),
              Input('my-date-picker-range', 'start_date'),
              Input('my-date-picker-range', 'end_date'),
              Input('stock-storage','data') )
def update_chart_time(start, end, stock):
    if stock is not None and end is not None and start is not None:
        df = pd.read_json(stock)
        df = df[start:end]#verificare per fare entrare giorno corrente
        df['adj close'] = df['5. adjusted close']
        df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
        df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
        df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])


        #calculate daily returns
        df['daily_return'] = df['adj close'].pct_change()
        df['log_returns'] = (np.log(df['adj close'] /    df['adj close'].shift(-1)))
        # calculate cumluative return
        df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000
    
        daily_returns = dcc.Graph(#id='daily-returns-graph',
                                        figure=px.histogram(df, x="daily_return",  title = 'Daily returns distribution'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Returns",
                                            yaxis_title="Frequency",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )   
        candlestick = dcc.Graph(#id='candlestick-graph',
                                        figure=go.Figure(data=[go.Candlestick(x=df.index,
                                                        open=df['adj open'],
                                                        high=df['adj high'],
                                                        low=df['adj low'],
                                                        close=df['adj close'])]
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        cum_returns = dcc.Graph(#id='cum-returns-graph',
                                        figure=px.line(df, x=df.index,y="cumluative_return", title = 'Cumulative returns'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Value of protfolio",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        daily_returns_time = dcc.Graph(#id='daily-returns-time-graph',
                                        figure=px.line(df, x=df.index,y="daily_return", title = 'Daily returns over time'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Returns",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )            
    
    
        return daily_returns, candlestick,  cum_returns, daily_returns_time
    else:
        return None,None,None,None


# Run app and display result inline in the notebook
app.run_server(mode="external", port = '8050' )
#mode="external", host="127.0.0.1", port=8060

env: ALPHAVANTAGE_API_KEY=NTN3D6P6LIMZ6A2Q#alpha advantage api key
Dash app running on http://127.0.0.1:8050/


In [12]:
df, meta_data = ts.get_daily_adjusted('SPOT', outputsize='full')
df.sort_index(ascending=True, inplace = True)  
df['adj close'] = df['5. adjusted close']
df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])
    
    
#calculate daily returns
df['daily_return'] = df['adj close'].pct_change()
# calculate cumluative return
df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000

In [46]:
import requests, re
url = "https://finance.yahoo.com/quote/KO/financials?p=KO"


source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

test = soup.find('div', class_='Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)')
    

AttributeError: 'NoneType' object has no attribute 'text'